# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [7]:
import pandas as pd
df = pd.read_csv('SMSSpamCollection', delimiter='\t', names = ['label', 'text'])
df.head()# Your code here

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [16]:
spam = df.query('label == "spam"')



In [12]:
ham = df.loc[df['label']=='ham']# Your code here

In [15]:
ham = ham.iloc[:747, :]

In [18]:
df = pd.concat([spam, ham], axis=0)

## Train-test split

Now implement a train-test split on the dataset: 

In [22]:
# Your code here
from sklearn.model_selection import train_test_split
X = df.text
y = df.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [26]:
freq_dict = {}

for c in y.unique():
    temp_df = train_df[train_df['label'] == c]
    bag = {}
    for row in temp_df.index:
        doc = temp_df['text'][row]
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
    freq_dict[c] = bag# Your code here

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [49]:
V = set()
for e in ['spam', 'ham']:
    for k, v in freq_dict[e].items():
        
        V.add(k)

In [50]:
len(V)

5966

## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [51]:
def bag_it(doc):
    bag = {}
    for word in doc.split():
        bag[word] = bag.get(word, 0) + 1
    return bag# Your code here

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [52]:
# Your code here
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in class_word_freq.keys():
        p = np.log(p_classes[class_])
        for word in bag.keys():
            num = bag[word]+1
            denom = class_word_freq[class_].get(word, 0) + V
            p += np.log(num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [56]:
p_classes = dict(df['label'].value_counts(normalize=True))
p_classes

{'spam': 0.5, 'ham': 0.5}

In [57]:
import numpy as np

y_hat_train = X_train.map(lambda x: classify_doc(X, freq_dict, p_classes, V))
residuals = y_train == y_hat_train
residuals.value_counts(normalize=True)# Your code here


AttributeError: 'Series' object has no attribute 'split'

## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!